In [ ]:
import kernels
import visualization
import affine
import numpy as np
from feature_detection import FeatureDetector
from utils import invert
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
radius = 3
spacing = 20
pixel_spacing = (0.5, 0.5, 0.5)

def fd_w_mismatched_kernel(radius, spacing, pixel_spacing, noise_mag, radius_mismatch):
    img = kernels.cylindrical_grid_intersection(pixel_spacing, radius*radius_mismatch, 1.2*spacing)
    img = np.dstack((img, img, img, img, img))
    
    middle = [(n - 1)/2 for n in img.shape]
    
    img[:, :, (middle[2] + int(1.5*radius/pixel_spacing[2])):] = 0.0
    img[:, (middle[1] + int(1.5*radius/pixel_spacing[1])):, :] = 0.0
    img[(middle[0] + int(1.5*radius/pixel_spacing[0])):, :, :] = 0.0
    
    img = invert(img)
    img = img - np.min(img)  # ensure it is positive
    
    img = img + noise_mag*np.random.rand(*img.shape)    
    
    ijk_to_xyz = np.array([
        [pixel_spacing[0], 0, 0, -middle[0]*pixel_spacing[0]],
        [0, pixel_spacing[1], 0, -middle[1]*pixel_spacing[1]],
        [0, 0, pixel_spacing[2], -middle[2]*pixel_spacing[2]],
        [0, 0, 0, 1],
    ])
    
    assert np.all(affine.apply_affine(ijk_to_xyz, np.array([middle]).T) == 0)
    
    fd = FeatureDetector('603A', 'mri', img, ijk_to_xyz)
    fd.grid_radius = radius
    fd.grid_spacing = spacing

    fd.run()
    return fd, middle, radius_mismatch

def plot_fds(result_tups):
    plt.figure(figsize=(12, 6))
    for fd, middle, radius_mismatch in result_tups:
        y = fd.feature_image[middle[0], middle[1], :]
        x = np.linspace(-1, 1, len(y))
        plt.plot(x, y, label='{}'.format(radius_mismatch))
    plt.legend()
    plt.grid()
    plt.savefig('fd1.2nn.pdf')

In [ ]:
results = [fd_w_mismatched_kernel(radius, spacing, pixel_spacing, 0, m) for m in [0.55, 0.7, 0.85, 1, 1.15, 1.3]]
plot_fds(results)

In [ ]:
visualization.slices_interactive(results[4][0].image)